 <div style="text-align: center; font-size: 35px">
    <b>
    Gradient descent
    </b>
</div>
<br/> <br/> <br/>

Chào các bạn, trong [bài viết trước](khanhxnguyen.com/deep-learning-4/), mình đã giới thiệu sơ lược về cách thức huấn luyện một model deep learning. 

Huấn luyện (training) gồm hai giai đoạn, forward và backward. Trong bước forward, ta tính loss function dựa vào dự đoán của model và label thật. Trong bước backward, ta tính gradient (đạo hàm) theo parameter $\theta$ của model, và cập nhật chúng theo quy tắc:
$$ \theta_{new} = \theta_{old} - \alpha \nabla_{\theta} L$$ với $\alpha$ là learning rate và $\nabla_{\theta}$ ký hiệu đạo hàm theo $\theta$ của loss function $L$.

Nhưng tại sao lại làm như vậy trong bước forward? Bài học hôm nay sẽ trả lời cho câu hỏi đó.

### Mục đích của huấn luyện

Mục đích của huấn luyện là tìm ra tham số $\theta^*$ tối ưu sao cho tổng loss function trên training set là nhỏ nhất. Theo ngôn ngữ toán học,

$$ \theta^* = \arg \min_{\theta} \mathcal{L}(\theta) = \arg \min_{\theta} \sum_{(x, y) \in D_{train}} L(f_{\theta}(x), y)$$ với $D_{train}$ là training set. 

Bài toán này là một dạng của **function optimization** (tối ưu hàm số). Ở đây vì $\theta$ không có điều kiện gì ràng buộc nên được gọi là **unconstrained optimization**. 

Vì không có công thức trực tiếp cho $\theta^*$, ta bắt buộc phải làm nhỏ dần $\mathcal{L}(\theta)$ qua nhiều bước. Ta bắt đầu với một $\theta$ ngẫu nhiên, và tìm cách làm cho $\theta$ càng ngày càng tiến gần tới $\theta^*$. Cách làm như vậy được gọi là một **iterative method**. Mỗi lần forward và backward chính là một bước biến đổi $\theta$ để làm $\mathcal{L}(\theta)$ nhỏ dần đi.

Nếu ai đã quen thuộc với **binary search** thì sẽ nhận ra thuật toán này cũng mang tư tưởng tương tự. Binary search thực chất là một dạng đặc biệt của function optimization với hàm được tối ưu chính là giá trị tuyệt đối giữa dự đoán hiện tại và giá trị cần tìm. 

### Gradient 

Có rất nhiều phương pháp  để tối ưu hàm số. Cách phương pháp sử dụng gradient (đạo hàm) được gọi là **first-order methods**. Nếu sử dụng đến hessian (đạo hàm bậc hai) được gọi là **second-order methods**. Cũng còn các phương pháp không sử dụng đến đạo hàm mà chỉ sử dụng đến hàm số được tối ưu được gọi là **zero-order methods**. First-order methods được ưa chuộng trong machine learning bởi vì chúng có độ phức tạp thấp hơn second-order methods nhưng lại tối ưu nhanh hơn zero-order methods.

**Gradient** là một *hàm trả về một vector*. Gradient của hàm $f$ theo $x$ kí hiệu là $\nabla_x f$. Mình tạm kí hiệu $g(x) = \nabla_x f $ để bạn đọc dễ nhìn ra đây là một hàm số. Gradient được tính bằng các công thức giống như tính đạo hàm một biến vậy nhưng áp dụng trên matrx hoặc tensor. Tuy nhiên, trong các framework deep learning hiện đại, gradient đều được tính tự động. 

**Nâng cao**: Kích thước (size) của gradient lúc nào cũng bằng với chiều của tham số được tính gradient theo. Ví dụ, nếu $x$ là vector 4 chiều thì gradient (theo $x$) trả về vector 4 chiều. Nếu $x$ là tensor $m \times n \times k$ thì gradient trả về tensor $m \times n \times k$. Mặc dù thật sự "gradient" lúc này không còn là gradient nữa (vì theo định nghĩa gradient chỉ trả về vector), nhưng với dân machine learning thì vẫn dùng từ "gradient" để chỉ "*hàm trả về tensor cùng kích thước với tham số mà dịch chuyển tham số theo tensor đó làm hàm số tăng/giảm nhanh nhất*".

**Lưu ý**: dù gradient là một hàm số, khi văn cảnh rõ ràng, mình sẽ dùng từ "gradient" để chỉ cho giá trị của hàm gradient tại một điểm nào đó của hàm số. Bạn cũng sẽ bắt gặp cách dùng này trong nhiều tài liệu khác.

### Tính chất của gradient

Tính chất ta quan tâm nhất về gradient đó là : **tại điểm $x = x_0$ nào đó, khi ta dịch chuyển $x_0$ theo hướng của $g(x_0)$ sẽ làm cho $f(x_0)$ tăng nhanh nhất**. Nói đơn giản hơn là:
$$ f(x + \alpha g(x)) \geq f(x) $$ với $\alpha$ là một hằng số dương đủ nhỏ. 

Tuy nhiên trong machine learning ta thường quan tâm đến việc *giảm* hơn là tăng hàm số. Vì thế ta thường đi ngược chiều với gradient:
$$ f(x - \alpha g(x)) \leq f(x) $$ 

**Câu hỏi**: *Tại sao đi ngược chiều gradient lại chắc chắn không tăng hàm số? Gợi ý: sử dụng tính chất $\nabla_x [-f] = -\nabla_x f$.*

Nếu $f(x^*)$ là điểm cực tiểu, hiển nhiên dịch chuyển hướng nào cũng làm tăng hàm số. Cho nên:
$$ f(x^* - \alpha g(x^*)) \geq f(x^*)$$

Kết hợp với tính chất của gradient, ta được:
$$ f(x^* - \alpha g(x^*)) = f(x^*)$$ 
vì *gradient tại điểm cực tiểu bằng vector 0*: 
$$g(x^*) = \vec{0}$$ 
Nói cách khác, khi hàm số đạt cực tiểu gradient sẽ bảo bạn ở yên đó và không đi đâu cả.

### Ví dụ minh họa bằng PyTorch

Giả sử vector $x$ có 4 chiều $x = (x_1, x_2, x_3, x_4)$. Ta muốn tối ưu hàm số:

$$ f(x) = x_1^2 + x_2^2 + x_3^2 + x_4^2 $$

Ta biết bằng hàm này đạt cực tiểu bằng 0 khi $x = \vec{0}$ nhưng hãy thử dùng iterative method để tìm nhé.

Đầu tiên ta tạo ra $x$ ngẫu nhiên là tính giá trị của $f(x)$.

In [88]:
import torch
from torch.autograd import Variable

torch.manual_seed(123)

# Đặt requires_grad=True để sau này tính đạo hàm.
x = Variable(torch.rand(4), requires_grad=True)
print x

Variable containing:
 0.6965
 0.7130
 0.2861
 0.4285
[torch.FloatTensor of size 4]



In [90]:
# f = sum_i(x_i^2)
f = (x*x).sum()
print f

Variable containing:
 1.2588
[torch.FloatTensor of size 1]



Bây giờ ta tính đạo hàm của $f$ theo $x$. Trong PyTorch, chỉ đơn giản gọi .backward().

In [91]:
f.backward()
g = x.grad
print g

Variable containing:
 1.3929
 1.4259
 0.5723
 0.8569
[torch.FloatTensor of size 4]



Nếu bạn nào rành về calculus thì biết rằng đạo hàm của $f(x)$ theo $x$ là $2x$. Kiểm chứng rằng $g$ đúng bằng $2x$. 

In [92]:
print 2*x - g

Variable containing:
 0
 0
 0
 0
[torch.FloatTensor of size 4]



Điều gì xảy ra nếu ta đi theo hướng của gradient. Hãy kiểm tra giá trị $f(x - g)$.

In [96]:
x_new = x - g
f_new = (x_new*x_new).sum()
print f_new
# Tính xem hàm số giảm được bao nhiêu so với giá trị cũ.
print f_new - f

Variable containing:
 1.2588
[torch.FloatTensor of size 1]

Variable containing:
 0
[torch.FloatTensor of size 1]



Thật kì lạ là hàm số không hề giảm mặc dù ta biết rõ rằng $f$ không phải đang ở vị trí cực tiểu vì $x \neq \vec{0}$. 

Vấn đề ở đây là ta đã chọn một $\alpha$ *quá lớn* ($\alpha = 1$). Ở đây, $x$ sẽ chỉ đổi dấu sau mỗi lần cập nhật như vậy ($x \rightarrow -x$). 

Tệ hơn nữa, nếu đặt $\alpha = 2$ thì hàm số có khả năng **diverge**, tức là sẽ lớn dần lên. Để thuận tiện ra định nghĩa một hàm để cập nhật hàm số.

In [97]:
def eval(alpha):
    x_new = x - alpha * g
    f_new = (x_new*x_new).sum()
    print f_new
    print f_new - f

eval(2)

Variable containing:
 11.3295
[torch.FloatTensor of size 1]

Variable containing:
 10.0707
[torch.FloatTensor of size 1]



Hãy thử với một $\alpha$ thật nhỏ. 

In [98]:
eval(0.25)

Variable containing:
 0.3147
[torch.FloatTensor of size 1]

Variable containing:
-0.9441
[torch.FloatTensor of size 1]



Hàm số đã giảm rồi! Ta thấy $\alpha$ đóng vai trò cực kì quan trọng trong việc tối ưu hàm số. Với hàm $f$ này, tồn tại một $\alpha$ tối ưu khiến cho hàm số lập tức dịch chuyển về cực tiểu.

In [99]:
eval(0.5)

Variable containing:
 0
[torch.FloatTensor of size 1]

Variable containing:
-1.2588
[torch.FloatTensor of size 1]



Ví dụ này có thể được mô phỏng bằng hình vẽ 2 chiều sau:

<img src="https://github.com/khanhptnk/deeplearning-tutorials/raw/master/images/5_gradient/example.png" width="50%">

### Gradient descent

Các thuật toán gradient descent (GD) đơn giản là việc lặp lại nhiều lần bước dịch chuyển tham số theo ngược chiều gradient, cũng chính là phương trình ở đầu bài. Tuy nhiên, mình thay hàm $L$ thành hàm $f_{\theta}$ bất kì. 

$$ \theta_{new} = \theta_{old} - \alpha \nabla_{\theta} f_{\theta}$$ với $\alpha$ được gọi là **learning rate** hoặc **step size**. 

Các thuật toán GD cơ bản khác nhau về cách cập nhật $\alpha$ theo thời gian. Learning rate không nên quá lớn hoặc quá nhỏ:
- Nếu quá lớn, hàm số có khả năng diverge.
- Nếu quả nhỏ, hàm số converge (hội tụ) rất chậm. 

Theo cảm tính, chúng ta cần phải giảm dần $\alpha$ vì càng gần điểm cực tiểu càng nhạy cảm, hàm số rất dễ diverge. 

Phiên bản đơn giản nhất của GD, để $\alpha$ là một hằng số không đổi, không được sử dụng nhiều trong machine learning. Phiên bản cải tiến hơn thì sử dụng một cách thủ công nào đó để giảm $\alpha$ theo thời gian, ví dụ $\alpha_t = \alpha_0 / (t + 1)$ với $t$ là số epoch. Các thuật toán adaptive GD hiện đại (AdaGrad, RMSProp, AdaDelta, Adam) tự động thay đổi $\alpha$ một cách hợp lý. Tuy nhiên, theo kinh nghiệm thực tế, vẫn cần phải kết hợp chúng với việc giảm $\alpha$ thủ công mới đạt hiệu quả cao nhất. 

**Câu hỏi**: *Khi nào cần giảm $\alpha$? Gợi ý: khi [overfitting](https://ml-book-vn.khanhxnguyen.com/1_2_overfitting.html) xảy ra*.

**Nâng cao**: GD không chắc chắn sẽ tìm ra được điểm cực tiểu của hàm số (global minimum), mà chỉ đảm bảo là nơi đó có gradient bằng 0. Bạn có thể hình dung việc tối ưu hàm số như là đi tìm thung lũng thấp nhất trong một vùng núi non. GD giống nhưng tổng lực của lực hấp dẫn và phản lực của mặt đất, sẽ kéo bạn lăn về nơi thấp hơn cho đến khi mặt đất không còn dốc nữa. Chỗ mặt đất không dốc có thể là (a) một trong nhiều thung lũng (local minimum) (b) cao nguyên (inflection point) (c) điểm yên ngựa (saddle point). 


<img src="https://github.com/khanhptnk/deeplearning-tutorials/raw/master/images/5_gradient/local.png" width="33%" align="left"> 

<img src="https://github.com/khanhptnk/deeplearning-tutorials/raw/master/images/5_gradient/inflection.png" width="33%" align="left">
<figure>
    <img src="https://github.com/khanhptnk/deeplearning-tutorials/raw/master/images/5_gradient/saddle.png" width="33%" align="left">
    <figcaption>Nguồn: <a href="https://en.wikipedia.org/wiki/Saddle_point">Wikipedia</a></figcaption>
</figure>



### Stochastic gradient descent

Gradient có tính chất là nếu $f = \sum_i f_i$ thì $\nabla f = \sum_i \nabla f_i$ (đạo hàm của tổng bằng tổng đạo hàm). Vì thế nếu 
$f_{\theta} = \mathcal{L}(\theta) = \sum_{(x, y) \in D_{train}} L(f_{\theta}(x), y)$, thì

$$\nabla_{\theta} f_{\theta} = \sum_{(x, y) \in D_{train}} \nabla_{\theta} L(f_{\theta}(x), y)$$

Do đó, mỗi lần muốn tính $\nabla_{\theta} \mathcal{L}(\theta)$ thì ta phải lặp qua toàn bộ training set và tính gradient cho từng cặp $(x, y)$. Làm thế rất mất thời gian cho một lần cập nhật $\theta$. Để cập nhật $\theta$ nhanh hơn, ta chỉ tính gradient cho các cặp $(x, y)$ trong một batch nhỏ để xấp xỉ gradient trên toàn set, tức là 

$$\nabla_{\theta} f_{\theta} \approx \sum_{(x, y) \in D_b} \nabla_{\theta} L(f_{\theta}(x), y)$$ với $D_b$ là một batch của $D_{train}$ có batch size bằng $b$.

Thuật toán xấp xỉ này gọi là "stochastic gradient descent". Mặc dù gradient này chỉ là xấp xỉ nhưng thực nghiệm cho thấy nó vẫn giúp model tiến bộ sau mỗi lần cập nhật. Tư tưởng của thuật toán này thật sự rất đơn giản: bạn muốn tính trung bình cộng của một đại lượng trên một dân số rất lớn (ví dụ tính trung bình chiều cao của người Việt Nam), bạn có thể xấp xỉ bằng cách sample (lấy mẫu) ra một tập dân số nhỏ rồi tính trung bình cộng trên đó; dĩ nhiên là làm một lần thì sẽ có sai số nhưng nếu sample nhiều lần thì sai số tự động sẽ triệt tiêu lẫn nhau. 

Ta cũng thấy là việc chia data thành batch ngoài việc giúp xấp xỉ đạo hàm trên toàn training set, còn giúp cho tính toán đạo hàm nhanh hơn bởi vì các cặp ví dụ trên cùng một batch sẽ được tính đạo hàm song song trên GPU (parallel).

### Đọc thêm:

Một số bài viết bằng tiếng Việt của blog "Machine Learning cơ bản":
[Gradient descent 1](http://machinelearningcoban.com/2017/01/12/gradientdescent/)
[Gradient descent 2](http://machinelearningcoban.com/2017/01/16/gradientdescent2/)
[Multivarate Calculus](http://machinelearningcoban.com/math/#-dao-ham-cua-ham-nhieu-bien)

So sánh giữa các phương pháp gradient descent của Sebastian Ruder: 
[An overview of gradient descent optimization algorithms](http://sebastianruder.com/optimizing-gradient-descent/)

Cách tính đạo hàm tự động minh họa bởi Chris Olah:
[Calculus on Computational Graphs: Backpropagation](http://colah.github.io/posts/2015-08-Backprop/)

Adam, một trong những optimizer thịnh hành nhất trong giới deep learning:
[ADAM: A METHOD FOR STOCHASTIC OPTIMIZATION](https://arxiv.org/pdf/1412.6980.pdf)